In [ ]:
# Install some required libraries with conda
!conda install -y -c conda-forge scikit-learn reportlab pdal geopandas pip jupyter geojson ipyvolume python-pdal rasterio

In [ ]:
import datetime
import geopandas as gpd
import glob
import json
import pdal
import pandas as pd
import os
import ipyvolume.pylab as p3
import matplotlib.cm
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import requests
import rasterio
import scipy
import shapely
import urllib.request

from pyproj import Proj, transform
from IPython.display import Image
from sklearn.cluster import DBSCAN

# Disable pandas copy on slice warning
pd.options.mode.chained_assignment = None

In [ ]:
# Finds the local maximum of a point cloud according to radius and point density threshold.
# Code from point cloud processing tutorial
# https://github.com/rockestate/point-cloud-processing/blob/master/notebooks/point-cloud-processing.ipynb
# More details in PCL
# http://docs.pointclouds.org/trunk/classpcl_1_1_local_maximum.html

def local_max(coords, radius, density_threshold=0):
    '''
    Find local maxima of points in a pointcloud.
    '''
    max_box = coords.copy()
    for i in range(2):
        max_box['X{}'.format(i)] = ((coords['X']/radius + i) /2).astype(int)*2
        max_box['Y{}'.format(i)] = ((coords['Y']/radius + i) /2).astype(int)*2
    max_box['X_'] = (coords['X']/radius).astype(int)
    max_box['Y_'] = (coords['Y']/radius).astype(int)
    for i in range(2):
        for j in range(2):
            max_box[str(i)+str(j)] = max_box.groupby(['X{}'.format(i), 'Y{}'.format(j)])['Z'].transform(np.max)
    density = max_box.groupby(['X_','Y_'])['Z'].transform(len)
    is_max = (max_box['00'] == max_box['10']) & (max_box['10'] == max_box['01']) & (max_box['01'] == max_box['11']) & (max_box['11'] == coords['Z'])
    
    return coords[is_max & (density >= (density_threshold))]

In [ ]:
NEARMAP_API_KEY = ''
PLANET_API_KEY = ''
LIDAR_FILENAME = os.path.join(os.getcwd(), 'RBG.las')
OUT_FOLDER = 'output'

In [ ]:
# JSON of the PDAL pipeline
pdal_pipeline = {
    "pipeline": [ LIDAR_FILENAME,
                 {   "type":"filters.pmf"},
                 {   "type":"filters.hag"}
                ]}
pipeline = pdal.Pipeline(json.dumps(pdal_pipeline))
pipeline.validate()
point_count = pipeline.execute()
lidar_df = pd.DataFrame(pipeline.arrays[0])
print("Number of points in LiDAR:", point_count)

In [ ]:
# Correct XYZ values relative to 0 for matplotlib to display
lidar_df['X_0'] = lidar_df['X']
lidar_df['Y_0'] = lidar_df['Y']
lidar_df['Z_0'] = lidar_df['Z']
lidar_df['X'] = lidar_df['X'] - lidar_df['X_0'].min()
lidar_df['Y'] = lidar_df['Y'] - lidar_df['Y_0'].min()
lidar_df['Z'] = lidar_df['Z'] - lidar_df['Z_0'].min()

In [ ]:
# Classify ground, grass and trees to determined values
grass_points = lidar_df.loc[(lidar_df['Classification'] == 1) & (lidar_df['HeightAboveGround'] < 0.7)]
ground_points = lidar_df.loc[lidar_df['Classification'] == 2]
tree_points = lidar_df.loc[(~lidar_df.index.isin(ground_points.index)) & (~lidar_df.index.isin(grass_points.index)) ]

In [ ]:
# Visualise LiDAR
fig = p3.figure(width=1000)

# Define all points and ground points for plot
tree_points_plot = p3.scatter(tree_points['Y'].values, tree_points['Z'].values, tree_points['X'].values, color='brown', size=.2)
ground_points_plot = p3.scatter(ground_points['Y'].values, ground_points['Z'].values, ground_points['X'].values, color='blue', size=.2)
grass_points_plot = p3.scatter(grass_points['Y'].values, grass_points['Z'].values, grass_points['X'].values, color='green', size=.2)
# Add labels to plot
fig.xlabel='Y'
fig.ylabel='Z'
fig.zlabel='X'

# Append both sets of points to plot
fig.scatters.append(tree_points_plot)
fig.scatters.append(ground_points_plot)
fig.scatters.append(grass_points_plot)
p3.squarelim()
p3.show()

In [ ]:
# Turn off ground points, cluster trees with DBSCAN
ground_points_plot.visible=False
grass_points_plot.visible=False

dbscan = DBSCAN(eps=1.2, min_samples=10).fit(tree_points[['X','Y','Z']].values)
tree_points['tree_idx'] = dbscan.labels_

print("Number of tree points:",len(tree_points))
print("Unique clusters:",len(tree_points['tree_idx'].unique()))

In [ ]:
# Generate colourmap for individual clusters
# https://www.robotswillkillusall.org/posts/mpl-scatterplot-colorbar.html
cmap = matplotlib.cm.get_cmap('flag')
normalize = matplotlib.colors.Normalize(vmin=min(tree_points['tree_idx']), vmax=max(tree_points['tree_idx']))
colors = [cmap(normalize(value)) for value in tree_points['tree_idx']]

# Plot clusters of points back onto scatter plot
classified_tree_points = p3.scatter(tree_points['Y'].values, tree_points['Z'].values, tree_points['X'].values, color=colors, size=.2)
tree_points_plot.visible=False
fig.scatters.append(classified_tree_points)

In [ ]:
# Find treetops, then place a sphere on each one
tree_tops = local_max(tree_points, radius=3, density_threshold=15)
tree_top_spheres = p3.scatter(tree_tops['Y'].values, tree_tops['Z'].values, tree_tops['X'].values, color='black', size=5, marker='sphere')

In [ ]:
# Find the tallest tree 
# This method assumes that all significant trees are the tallest in the LiDAR
# Alternatively, the treetop closest to the centre tree could be used
tree_top_spheres.visible = False
tallest_tree = pd.DataFrame(tree_tops.loc[tree_tops['Z'].idxmax()]).T
tallest_tree_sphere = p3.scatter(tallest_tree['Y'].values, tallest_tree['Z'].values, tallest_tree['X'].values, color='green', size=5, marker='sphere')

In [ ]:
tallest_tree_lidar_df = tree_points.loc[tree_points['tree_idx'] == 3]
print("Number of points in tallest tree:", len(tallest_tree_lidar_df))

In [ ]:
# Plot only the tallest tree's LiDAR
tallest_tree_lidar_points = p3.scatter(tallest_tree_lidar_df['Y'].values, tallest_tree_lidar_df['Z'].values, tallest_tree_lidar_df['X'].values, color='green', size=.2)
classified_tree_points.visible=False
fig.scatters.append(tallest_tree_lidar_points)

In [ ]:
# Volume and height of tree cloud (in cubic metres, expecting coordinate system is metres - UTM WGS84 in this case)
tree_volume = round(scipy.spatial.ConvexHull(tallest_tree_lidar_df[['X','Y','Z']]).volume,2)
tree_height = round(tallest_tree['HeightAboveGround'].values[0],2)
print("Tree dimensions\n---------------\nVolume:", str(tree_volume)+"m^3", "\nHeight:", str(tree_height)+"m")

# Nearmap imagery

In [ ]:
# X Y Z values of tallest tree in GDA94
tallest_tree

In [ ]:
def reproject_xy(from_crs, to_crs, x, y):
    inProj = Proj(init='epsg:'+str(from_crs))
    outProj = Proj(init='epsg:'+str(to_crs))
    
    return transform(inProj,outProj,x,y)

In [ ]:
# Reproject coordinates from LiDAR into latitude and longitude

project_crs = 28355 # GDA94 Z55
nearmap_crs = 4326 # WGS 84

lon, lat = reproject_xy(project_crs, nearmap_crs, tallest_tree['X_0'].values[0], tallest_tree['Y_0'].values[0])

print("Tree's lat/long:", str(lat) + ', ' + str(lon))

In [ ]:
# Collect todays date for latest Nearmap imagery
todays_date = datetime.datetime.today().strftime('%Y%m%d')

# Form a Nearmap API query
# API docs for imagery: https://docs.nearmap.com/display/ND/Image+API
nearmap_url = "http://au0.nearmap.com/staticmap?center="+str(y2)+","\
                +str(x2)+"&size=3000x3000&zoom=22&date="+todays_date\
                +"&httpauth=false&apikey="+NEARMAP_API_KEY

# Retrieve data and save as a local file
# URL Retrieve https://docs.python.org/3.0/library/urllib.request.html
nm_filename, nm_url = urllib.request.urlretrieve(nearmap_url, os.path.join(os.getcwd(), 'nearmap.jpg'))

In [ ]:
# Show nearmap image in notebooks
Image(filename='nearmap.jpg') 

# Planet multispectral data

In [ ]:
# Create convex hull around tree points, then create bounding box for Planet imagery acquisition
tree_polygon = shapely.geometry.MultiPoint(tallest_tree_lidar_df[['X_0','Y_0']].values).convex_hull

# Reproject coordinates to WGS84 (suitable for Planet)
tree_bbox = [reproject_xy(project_crs, nearmap_crs, tree_polygon.bounds[0], tree_polygon.bounds[1]),
            reproject_xy(project_crs, nearmap_crs, tree_polygon.bounds[2], tree_polygon.bounds[3])]
tree_bbox_wgs84 = shapely.geometry.box(tree_bbox[0][0], tree_bbox[0][1], tree_bbox[1][0], tree_bbox[1][1])
tree_bbox_list = list(bbox_wgs84.exterior.coords)

In [ ]:
# This code has been adapted from the Planet API resource notebooks at:
# https://github.com/planetlabs/notebooks/blob/master/jupyter-notebooks/data-api-tutorials/planet_data_api_introduction.ipynb
# as well as their NDVI method at:
# https://developers.planet.com/tutorials/calculate-ndvi/

# Helper function to printformatted JSON using the json module
def p(data):
    print(json.dumps(data, indent=2))
    
# Function to assist with downloading Planet data
def pl_download(url, folder, api_key, filename=None):
    # Send a GET request to the provided location url, using your API Key for authentication
    res = requests.get(url, stream=True, auth=(api_key, ""))
    # If no filename argument is given
    if not filename:
        # Construct a filename from the API response
        if "content-disposition" in res.headers:
            filename = res.headers["content-disposition"].split("filename=")[-1].strip("'\"")
        # Construct a filename from the location url
        else:
            filename = url.split("=")[1][:10]
            
    if not os.path.isdir(os.path.join('.',folder)):
        os.makedirs(os.path.join('.',folder))
        print ("Created folder", os.path.join('.',folder))
    if not os.path.isfile(os.path.join('.',folder,filename)):
        print("Downloading...")
        with open(os.path.join('.',folder, filename), "wb") as f:
            for chunk in res.iter_content(chunk_size=1024):
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)
                    f.flush()
        print(os.path.join('.',folder,filename), "is downloaded")
        print("Creating NDVI and .png preview")
        form_ndvi(filename, folder)
    else:
        print(os.path.join('.',folder,filename),"already exists")
   
    return filename    

# Takes the input features list and returns a list of date, id and cloud cover
def concat_feature_list(features, bbox_polygon):
    feature_list = []
    for f in features:
        # Build bbox with Shapely to check if it fits our project's geometry
        feature_bbox = shapely.geometry.Polygon(f['geometry']['coordinates'][0])
        date = f["properties"]['acquired'].split(':')[0].split('T')[0].split('-')
        feature_list.append([date[2], date[1], date[0], f['id'], 
                             f['properties']['cloud_cover'], f['_links']['assets'], f['properties']['sun_azimuth'], f['properties']['sun_elevation'], f['properties']['view_angle'], feature_bbox.contains(bbox_polygon)])
    return feature_list

def acquire_planet_data(feature_asset_url, OUTPUT_DIR, session, api_key):
    print("Working on", feature_asset_url)
    
    # Get the assets link for the item
    assets_url = feature_asset_url
    # Send a GET request to the assets url for the item (Get the list of available assets for the item)
    res = session.get(assets_url)
    # Assign a variable to the response
    assets = res.json()
    # We want the analytic datatype
    analytic = assets["analytic"]

    # Setup the activation url for a particular asset (in this case the visual asset)
    activation_url = analytic["_links"]["activate"]

    # Send a request to the activation url to activate the item
    res = session.get(activation_url)

    # Print the response from the activation request

    if res.status_code == 202:
        print("202 - The request has been accepted and the activation will begin shortly.")
    elif res.status_code == 204:
        print("204 - The asset is already active and no further action is needed.")
    elif res.status_code == 401:
        print("401 - The user does not have permissions to download this file.")
    else:
        print("Undefined error.")

    asset_activated = False

    while asset_activated == False:
        # Send a request to the item's assets url
        res = session.get(assets_url)

        # Assign a variable to the item's assets url response
        assets = res.json()

        # Assign a variable to the visual asset from the response
        analytic = assets["analytic"]

        asset_status = analytic["status"]

        # If asset is already active, we are done
        if asset_status == 'active':
            asset_activated = True
            print("Asset is active and ready to download")

    # Assign a variable to the visual asset's location endpoint
    location_url = analytic["location"]
    # Download the file from an activated asset's location url
    filename = pl_download(location_url, OUTPUT_DIR, api_key)
    
def get_planet_data(OUTPUT_DIR, boundingbox_list, bbox_polygon, api_key):
    print("\n------- Obtaining Planet Data & Calculating NDVI -------")
    # Setup Planet Data API base URL
    URL = "https://api.planet.com/data/v1"

    # Setup the session
    session = requests.Session()

    # Authenticate
    session.auth = (api_key, "")

    # Specify the sensors/satellites or "item types" to include in our results, 
    # date range, cloud cover and bounding box
    item_types = ["PSScene4Band"]

     # Set from 2 years from today
    thisyear = datetime.datetime.today().strftime('%Y')
    thismonth = datetime.datetime.today().strftime('%m')
    thisday = datetime.datetime.today().strftime('%d')
    thistime = 'T00:00:00Z'
    planet_start = str(int(thisyear)-2)+'-'+thismonth+'-'+thisday+thistime
    planet_end = thisyear+'-'+thismonth+'-'+thisday+thistime
    
    date_filter = {
      "type": "DateRangeFilter",
      "field_name": "acquired",
      "config": {
        "gt": planet_start,
        "lte": planet_end
      }
    }

    range_filter = {
      "type": "RangeFilter",
      "field_name": "cloud_cover",
      "config": {
        "lt": 0.1
      }
    }

    RBG_geometry_filter = {
      "type": "GeometryFilter",
      "field_name": "geometry",
      "config": {
        "type": "Polygon",
        "coordinates": [
          boundingbox_list
        ]
      }
    }

    # Combine the filters together in the config item
    and_filter = {
        "type": "AndFilter",
        "config": [range_filter, RBG_geometry_filter, date_filter]
    }
    
    # Make a GET request to the Planet Data API
    res = session.get(URL)
    
    # Setup the stats URL
    stats_url = "{}/stats".format(URL)

    # Construct the request.
    request = {
        "item_types" : item_types,
        "interval" : "month",
        "filter" : and_filter
    }
    # Send the POST request to the API stats endpoint
    res = session.post(stats_url, json=request)
    
    # Setup the quick search endpoint url
    quick_url = "{}/quick-search".format(URL)

    request = {
        "item_types" : item_types,
        "interval" : "month",
        "filter" : and_filter
    }

    # Send the POST request to the API quick search endpoint
    res = session.post(quick_url, json=request)

    # Assign the response to a variable
    geojson = res.json()
    features = geojson["features"]

    # Loop over all the features from the response
    suitable_images = pd.DataFrame(concat_feature_list(features, bbox_polygon), 
                                   columns=['day', 'month', 'year', 'id', 'cloud', 'asset_url', 'sun_azimuth', 'sun_elevation', 'view_angle', 'fits_project'])
    print("Features:", len(features), "Total features:", len(suitable_images))

    # If a page reaches its maximum, select the next page and concatenate results until all results are found
    while len(features) == 250:
        next_url = geojson["_links"]["_next"]
        res = session.get(next_url)
        geojson = res.json()
        features = geojson["features"]
        suitable_images = suitable_images.append(pd.DataFrame(concat_feature_list(features, bbox_polygon), 
                                                              columns=['day', 'month', 'year', 'id', 'cloud', 'asset_url', 'sun_azimuth', 'sun_elevation', 'view_angle', 'fits_project']))
        print("Features:", len(features), "Total features:", len(suitable_images))
    print("Finished loading results")
    
    # Reset the dataframe index after appending
    suitable_images = suitable_images.reset_index()

    # Clean out images that don't fit within the boundary of our project
    #suitable_images = suitable_images.loc[suitable_images['fits_project'] == True]

    # Find the day of each month over the data series, where the cloud cover is the least

    df1 = suitable_images.loc[suitable_images.groupby(['year', 'month'])['cloud'].idxmin()].set_index(['year', 'month'])
    df2 = suitable_images.loc[suitable_images.groupby(['year', 'month'])['cloud'].idxmax()].set_index(['year', 'month'])

    unique_days = pd.concat([df1, df2], axis=1, keys=('min','max'))
    unique_days.columns = unique_days.columns.map('_'.join)
    
    # Should be ready to download... Go!
    for asset_url in unique_days['min_asset_url']:
        acquire_planet_data(asset_url, OUTPUT_DIR, session, api_key)
    
    satellite_data = unique_days
    print("satellite_data")
    print(satellite_data)
    satellite_data['year'], satellite_data['month'] = satellite_data.index[0]
    satellite_data = satellite_data[['year', 'month','min_day','min_cloud', 'min_sun_azimuth','min_sun_elevation','min_view_angle']]
    satellite_data = satellite_data.rename(columns={'min_day': 'day', 
                                                    'min_cloud': 'cloud_cover',
                                                    'min_sun_azimuth': 'sun_azimuth',
                                                    'min_sun_elevation': 'sun_elevation',
                                                    'min_view_angle': 'view_angle'})

    return satellite_data

def form_ndvi(filename, folder):
     # The following NDVI calculation method is adapted from https://developers.planet.com/tutorials/calculate-ndvi/
    with rasterio.open(os.path.join('.',folder,filename)) as src:
        band_red = src.read(3)
    with rasterio.open(os.path.join('.',folder,filename)) as src:
        band_nir = src.read(4)

    # Allow division by zero
    np.seterr(divide='ignore', invalid='ignore')

    # Calculate NDVI
    ndvi = (band_nir.astype(float) - band_red.astype(float)) / (band_nir + band_red)

    # Set spatial characteristics of the output object to mirror the input
    kwargs = src.meta
    kwargs.update(
        dtype=rasterio.float32,
        count = 1)

    # Create the file
    with rasterio.open(os.path.join('.',folder,filename+'.NDVI.tif'), 'w', **kwargs) as dst:
            dst.write_band(1, ndvi.astype(rasterio.float32))

    # Export as PNG 
    plt.imsave(os.path.join('.',folder,filename+'ndvi_cmap.png'), ndvi)
    
def crop_ndvi(shapefile, rasterin, rasterout, folder):
    outfolder = os.path.join('.',folder,'ndvi')
    
    # Create NDVI folder
    if not os.path.isdir(outfolder):
        os.makedirs(outfolder)
        print ("Created folder", outfolder)
        
    with fiona.open(shapefile, "r") as shapefile:
        shpfeatures = [feature["geometry"] for feature in shapefile]
        
    with rasterio.open(os.path.join('.',folder,rasterin), "r") as src:
        out_image, out_transform = rasterio.mask.mask(src, shpfeatures, crop=True)
        out_meta = src.meta.copy()
        
    out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})
    
    with rasterio.open(os.path.join(outfolder,rasterout), "w", **out_meta) as dest:
        dest.write(out_image)
        
def crop_the_planet(outdir):
    print("\n------- Cropping Planet Data -------")
    for tif in glob.iglob(os.path.join(os.getcwd(),outdir,'*NDVI.tif')):
        outtif = os.path.join(tif.split('/')[-1:][0].split('_')[0][:-2] + '.ndvi.tif')
        crop_ndvi(boundary_shapefile, tif, outtif, OUTPUT_DIR)
        print(tif.split('/')[-1].split('_')[0][:-2], "- Cropped NDVI:", outtif)

    for tif in glob.iglob(os.path.join(os.getcwd(),outdir,'ndvi','*.tif')):
        with rasterio.open(tif) as src:
            ndvi = src.read(1)
        png_filename = os.path.join('.',outdir,'ndvi',tif.split('.')[0]+'ndvi_cmap.png')
        plt.imsave(png_filename, ndvi)
        print(tif.split('/')[-1].split('.')[0], "- PNG file output:", png_filename.split(' ')[-1].split('/')[-1])

In [ ]:
# Request planet data, then download
get_planet_data(OUT_FOLDER, tree_bbox_list, tree_bbox_wgs84, PLANET_API_KEY)

In [ ]:
# Crop the NDVI to the boundary of the RBG. This won't work without creating a shapefile
# of the RBG boundary. 

#crop_the_planet(OUT_FOLDER)

# Notes
* Be aware that Planet are changing their API rules from the 15th of May. Refer to their API documentation for updates https://developers.planet.com/docs/api/
* Access to the original code can be found on my Github page. Any changes will require some editing. I cannot distribute the LiDAR data. The code was written very ad-hoc and is not illustrative of best practices. https://github.com/evanjt/lidar-trees

# References
## Tutorial adapted from several tutorials and resources listed below

* https://github.com/rockestate/point-cloud-processing/blob/master/notebooks/point-cloud-processing.ipynb
* http://docs.pointclouds.org/trunk/classpcl_1_1_local_maximum.html
* https://developers.planet.com/tutorials/calculate-ndvi/
* https://github.com/planetlabs/notebooks/blob/master/jupyter-notebooks/data-api-tutorials/planet_data_api_introduction.ipynb
* https://www.robotswillkillusall.org/posts/mpl-scatterplot-colorbar.html
* Royal Botanic Gardens, Melbourne & MUASIP @ The University of Melbourne